#### Listing down the procedure for this approach:

1. Load: Load the data, in this case a pdf through PyPDFLoader
2. Split into Chunks: PDf document is splitted into short chunks
3. Embed: Each chunks of section is embedded with the OpenAI API
4. Save: Embeddings are saved 

#### Method 1: With using Langchains

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from openai import OpenAI
from langchain_text_splitters import CharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


loader = PyPDFLoader("ConceptsofBiology-WEB.pdf")

index = VectorstoreIndexCreator(
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0),
    embedding=OpenAIEmbeddings(),
    vectorstore_cls=Chroma
).from_loaders([loader])

query = "The smallest unit of biological structure that meets the functional requirements of “living” is the?"
index.query(llm=OpenAI(), question=query, chain_type="stuff")

#### Method 2: Without using Langchains

In [ ]:
import fitz
import ast 
from openai import OpenAI
import pandas as pd  
import tiktoken  
from scipy import spatial 


input_file = r"ConceptsofBiology-WEB.pdf"

file_handle = fitz.open(input_file)
 
lst = []
i = 17
while i < 38:
    page = file_handle[i]
    i = i+1
    lst.append(i)
print(lst) 

text = "" 
for i in lst: 
    page = file_handle[i]   
    text+=page.get_text() 
print(text) 

In [ ]:
# split the documents into chunks
K = 1000
 
chnk_len = len(text) // K
 
res = []
for idx in range(0, len(text), chnk_len):
     
    # appending sliced string
    res.append(text[idx : idx + chnk_len])

print("The K chunked list : " + str(res))

In [ ]:
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "OpenAI API key"))

query = f"""Use the text content below to answer the subsequent question. If the answer cannot be found, write "I don't know."

text:
\"\"\"
{text}
\"\"\"

Question: The smallest unit of biological structure that meets the functional requirements of “living” is the?"""

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the smallest unit of biological structure.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)
print(response.choices[0].message.content)                                                                     

In [ ]:
ask('A suggested and testable explanation for an event is called a ?', model="gpt-4")